<a href="https://colab.research.google.com/github/StevenTrivino/Specializacion/blob/main/lesson1-intro/intro_to_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow

** ¡Bienvenid@!**

Este cuaderno es parte del curso de introducción a TensorFlow. Si has tenido experiencia previa con Jupyter, este ambiente te será muy familiar. Esto es Colaboratory, un ambiente de cuadernos de Python. En los cuadernos de Python hay celdas que tienen texto (Markdown, que nos permite darle mucho estilo) y celdas en las que hay código de Python.

Puedes acceder a este cuaderno en [Colab](https://colab.research.google.com/drive/1TIIaFJnnQsL_BwwJQAjU-JkZvaRJ-ybC).

Para introducir, Colab nos da mucha flexibilidad y nos facilitará el curso eliminando la necesidad de configuración e instalación en su sistema. [Colab](https://medium.com/tensorflow/colab-an-easy-way-to-learn-and-use-tensorflow-d74d1686e309) es gratuito y es de Google. Más adelante veremos que también nos da acceso gratuito a hardware como GPUs y TPUs. Colab ya tiene muchas herramientas, como TensorFlow, instaladas. Si has llegado aquí, probablemente estés accediendo a una copia de un cuaderno de GitHub. 

## Introducción

Como se mencionó antes, Colab ya tiene TensorFlow instalado. En esta sección iniciamos importando la librería.

In [ ]:
import tensorflow as tf

En TensorFlow todo es un **grafo**. Luego veremos Eager Execution, pero con el modelo de grafos, TensorFlow separa la definición de las operaciones o computaciones de su ejecución. La idea es que primero se define el grafo y luego se ejecuta en una sesión. TensorFlow sigue un modelo de programación típico para programación paralela llamado DataFlow. En un grafo, los nodos representan unidades de computación y las aristas representan los datos consumidos o producidos. En otras palabras:

*   Un nodo es una operación (`tf.Operation`). También puede tener variables y constantes, las cuales veremos pronto.
*   Una arista son datos (`tf.Tensor`)

Un **tensor** es un arreglo de n dimensiones. Un tensor de 0 dimensiones es un escalar (un número). Un tensor de 1 dimensión es un vector. Finalmente, un tensor de n dimensiones es una matriz.  El número de dimensiones es conocido como **rank**.

En la siguiente celda se tiene una operación, [tf.add](https://www.tensorflow.org/api_docs/python/tf/math/add), que suma dos tensores (de 0 dimensiones). Más adelante veremos cómo visualizar los grafos con TensorBoard, pero, por ahora, ponemos aquí la imagen.

![alt text](https://github.com/AILearnersMX/TensorFlow-Course/blob/master/lesson1-intro/add.png?raw=true)

In [ ]:
a = tf.add(3, 5)
print(a)

Tensor("Add_3:0", shape=(), dtype=int32)


Como se mencionó antes, el grafo está compuesto por operaciones, **ops**, las cuales reciben como entrada cero o más tensores o pueden generar nuevos tensores.

Al utilizar `print`, no nos está desplegando el resultado esperado. En cambio, sólo imprime información del tensor. Se debe crear una **session** desde la cuál se ejecutará el grafo. En otras palabras, el código de antes sólo genera el grafo que determina los tamaños de los tensores y las operaciones que se ejecutarán dentro de él. Para que los valores fluyan a través del grafo, se debe hacer con una sesión.

In [ ]:
sess = tf.Session()
print(sess.run(a))
sess.close()

8


In [ ]:
with tf.Session() as sess:
  print(sess.run(a))

8


## Ventajas de grafo

En la siguiente celda hacemos la siguiente operación:

`(2*3)^(2+5)`

![alt text](https://github.com/AILearnersMX/TensorFlow-Course/blob/master/lesson1-intro/complex_graph.png?raw=true)

In [ ]:
x = 2
y = 3

op1 = tf.add(x, y)         
op2 = tf.multiply(x, y)    
op3 = tf.pow(op2, op1)

with tf.Session() as sess:
  print(sess.run(op3))

7776


Un grafo muestra las dependencias entre las operaciones. Esto evita ejecutar código innecesario. En la siguiente celda, tenemos una operación inutil, `useless`, que no se utiliza dentro de la sesión. Gracias a poder representar el programa como un grafo, esa operación nunca se ejecuta.

![alt text](https://github.com/AILearnersMX/TensorFlow-Course/blob/master/lesson1-intro/useless_graph.png?raw=true)

In [ ]:
x = 2
y = 3

op1 = tf.add(x, y)         
op2 = tf.multiply(x, y)   
op3 = tf.pow(op2, op1)
useless = tf.multiply(x, op1)

with tf.Session() as sess:
  print(sess.run(op3))

7776


La siguiente celda muestra cómo ejecutar más de una operación dentro de la sesión. Esta es una buena oportunidad para aprender a buscar en la documentación oficial. [tf.Session().run(...)](https://www.tensorflow.org/api_docs/python/tf/Session#run) tiene varios parámetros. El que nos interesa es `fetches`.

Como dice la documentación, el método `run` ejecuta el fragmento del grafo necesario para ejecutar esa operación. Viendo la documentación, `sess.run` puede recibir una lista de elementos de grafo como tensores y operaciones. Aquí un ejemplo:

In [ ]:
x = 2
y = 3

op1 = tf.add(x, y)         
op2 = tf.multiply(x, y)   
op3 = tf.pow(op2, op1)
op4 = tf.multiply(x, op1)

with tf.Session() as sess:
  z, not_useless = sess.run([op3, op4])
  print(z, not_useless)

7776 10


##Paralelización

La siguiente celda nos permite determinar si hay un GPU disponibe. Por suerte, Colab nos da GPU y TPU gratuitos (con algunas limitaciones). En la barra de herramientas, podemos agregar GPU en: Runtime > Change Runtime Type > Hardware Acceleration > GPU. Nota que esto cambia a otro ambiente, por lo que tendrás que volver a correr las celdas para importar TensorFlow.

In [ ]:
print(tf.test.is_gpu_available())
print(tf.test.gpu_device_name())

True
/device:GPU:0


En la siguiente celda hacemos algo llamado explicit device placement. Esto significa que podemos ejecutar algo directamente en un pedazo de hardware específico. En este ejemplo se multiplican dos tensores constantes. 

In [ ]:
with tf.device('/gpu:0'):
  a = tf.constant([1.0, 2.0, 3.0], name='a')
  b = tf.constant([1.0, 2.0, 5.0], name='b')
  c = tf.multiply(a, b)
  
  
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
print(sess.run(c))

[ 1.  4. 15.]


## Eager Execution

Eager Execution es un modelo imperativo de TensorFlow que evita tener que ejecutar sesiones y crear grafos. En un taller más adelante entraremos en mayor detalle, pero mostraremos su funcionamiento en alto nivel.

Con Eager Execution, el grafo se ejecuta directamente. Sus ventajas son:

*   Más intuitivo y fácil de aprender. Funciona más parecido al código de Python normal.
*   Más fácil de hacer debugging en este modelo.
*   Reduce código boilerplate.
*   Flujo más sencillo.

Pero también tiene desventajas:

*   No funciona tan bien al distribuirlo.
*   Peor desempeño en producción

¡No se preocupen! Ambas técnicas se pueden mezclar. Se puede convertir código de Eager Execution a grafo sin dificultad, pero esto lo veremos más adelante. Es importante destacar que en TensorFlow 2.0, Eager Execution será central y estará habilidado por defecto. Antes de correr la siguiente celda, es necesario que reinicies el ambiente (Runtime > Restart Runtime ).

In [ ]:
import tensorflow as tf
tf.enable_eager_execution()

In [ ]:
tf.executing_eagerly() 

True

In [ ]:
print(tf.add([1, 2], [2, 1]))

tf.Tensor([3 3], shape=(2,), dtype=int32)


In [ ]:
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))
print(tf.encode_base64("hello world"))

tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(b'aGVsbG8gd29ybGQ', shape=(), dtype=string)


In [ ]:
x = tf.matmul([[1]], [[2, 3]])
print(x.shape)
print(x.dtype)

(1, 2)
<dtype: 'int32'>


## NumPy y TensorFlow

Cerramos el cuaderno hablando de la compatibilidad entre NumPy y TensorFlow. A diferencias de los `ndarray`s, `Tensor` es inmutable y tiene accesso a memoria acelerada (GPU). Aún así, trabajar con NumPy y TensorFlow es extremadamente sencillo y podemos utilizar ambos en un proyecto (es lo normal).

In [ ]:
import numpy as np

many_ones = np.ones([3, 3])

Cualquier op que reciba un `ndarray` lo convertirá directamente a un tensor.

In [ ]:
tensor = tf.multiply(many_ones, 42)
print(tensor)

tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)


A su vez, las operaciones de NumPy pueden recibir un tensor directamente.

In [ ]:
print(np.add(tensor, 1))

[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]


In [ ]:
print(type(tensor))

<class 'tensorflow.python.framework.ops.EagerTensor'>
